The file below has two parts as below. 
- Take a sample min-vertex-network graph to calculate the cost hamiltoniann
- Add noise to the circuit, to calculate the a modified cost and check if in expected range of values.

https://pennylane.ai/challenges/qaoa/


In [76]:
import networkx as nx
from pennylane import qaoa
from pennylane import numpy as np
import pennylane as qml
from matplotlib import pyplot as plt
import torch

edges = [(0, 1), (1, 2), (2, 0), (2, 3)]
graph = nx.Graph(edges)
num_wires= 4

dev = qml.device('default.mixed', wires = num_wires)

cost_h, mixer_h = qaoa.min_vertex_cover(graph, constrained=False)

print("Cost Hamiltonian", cost_h)
print("Mixer Hamiltonian", mixer_h)

def qaoa_layer(gamma, alpha):
    qaoa.cost_layer(gamma, cost_h)
    qaoa.mixer_layer(alpha, mixer_h)

wires = range(4)
depth = 2

def circuit(params, **kwargs):
    for w in wires:
        qml.Hadamard(wires=w)
    qml.layer(qaoa_layer, depth, params[0], params[1])

dev = qml.device("default.mixed", wires=wires)

@qml.qnode(dev)
def cost_function(params):
    circuit(params)
    return qml.expval(cost_h)

optimizer = qml.GradientDescentOptimizer()
steps = 100
params = np.array([[0.5, 0.5], [0.5, 0.5]], requires_grad=True)

for i in range(steps):
    params = optimizer.step(cost_function, params)

print("Optimal Parameters")
print(params, cost_function(params))



Cost Hamiltonian   (-0.25) [Z3]
+ (0.5) [Z0]
+ (0.5) [Z1]
+ (1.25) [Z2]
+ (0.75) [Z0 Z1]
+ (0.75) [Z0 Z2]
+ (0.75) [Z1 Z2]
+ (0.75) [Z2 Z3]
Mixer Hamiltonian   (1) [X0]
+ (1) [X1]
+ (1) [X2]
+ (1) [X3]
Optimal Parameters
[[0.59805356 0.94199357]
 [0.52797309 0.85553493]] -1.5720009432822621


In [77]:
edges = [(0, 1), (1, 2), (2, 0), (2, 3)]
num_wires = 4

# We define the Hamiltonian for you!

ops = [qml.PauliZ(0), qml.PauliZ(1),qml.PauliZ(2), qml.PauliZ(3), qml.PauliZ(0)@qml.PauliZ(1), qml.PauliZ(0)@qml.PauliZ(2),qml.PauliZ(1)@qml.PauliZ(2),qml.PauliZ(2)@qml.PauliZ(3)]
coeffs = [0.5, 0.5, 1.25, -0.25, 0.75, 0.75, 0.75, 0.75]

cost_hamiltonian = qml.Hamiltonian(coeffs, ops)

# Write any helper functions you need here
true_value = -1.5720009432822621

@qml.qnode(dev) 
def qaoa_circuit(params, noise_param):

    """
    Define the noisy QAOA circuit with only CNOT and rotation gates, with Depolarizing noise
    in the target qubit of each CNOT gate.

    Args:
        params(list(list(float))): A list with length equal to the QAOA depth. Each element is a list that contains 
        the two QAOA parameters of each layer.
        noise_param (float): The noise parameter associated with the depolarization gate

    Returns: 
        (np.tensor): A numpy tensor of 1 element corresponding to the expectation value of the cost Hamiltonian
    
    """

    for i in range(num_wires):
        qml.RY(np.pi/2, i)
        # qml.Hadamard(i)
    for param in params: 
        gamma, alpha = param 

        qml.RZ(2*gamma*0.5, wires=0)
        qml.RZ(2*gamma*0.5, wires=1)
        qml.RZ(2*gamma*1.25, wires=2)
        qml.RZ(2*gamma*-0.25, wires=3)

        qml.CNOT(wires=[0,1])
        qml.DepolarizingChannel(noise_param, wires=1)
        qml.RZ(2*gamma*0.75, 1)
        qml.CNOT(wires=[0,1])
        qml.DepolarizingChannel(noise_param, wires=1)

        qml.CNOT(wires=[0,2])
        qml.DepolarizingChannel(noise_param, wires=2)
        qml.RZ(2*gamma*0.75, 2)
        qml.CNOT(wires=[0,2])
        qml.DepolarizingChannel(noise_param, wires=2)

        qml.CNOT(wires=[1,2])
        qml.DepolarizingChannel(noise_param, wires=2)
        qml.RZ(2*gamma*0.75, 2)
        qml.CNOT(wires=[1,2])    
        qml.DepolarizingChannel(noise_param, wires=2)

        qml.CNOT(wires=[2,3])
        qml.DepolarizingChannel(noise_param, wires=3)
        qml.RZ(2*gamma*0.75, 3)
        qml.CNOT(wires=[2,3])                    
        qml.DepolarizingChannel(noise_param, wires=3)

        for i in range(num_wires):
            qml.RX(2*alpha, wires=i)      
            
        return qml.expval(cost_hamiltonian)
def approximation_ratio(qaoa_depth, noise_param):
    """
    Returns the approximation ratio of the QAOA algorithm for the Minimum Vertex Cover of the given graph
    with depolarizing gates after each native CNOT gate

    Args:
        qaoa_depth (float): The number of cost/mixer layer in the QAOA algorithm used
        noise_param (float): The noise parameter associated with the depolarization gate
    
    Returns: 
        (float): The approximation ratio for the noisy QAOA
    """

    print ("approximation_ratio : ", qaoa_depth, noise_param)
    params = np.random.uniform(0, 2*np.pi, (qaoa_depth, 2))

    def cost_function(p):
        return qaoa_circuit(p, noise_param)
    
    opt = qml.AdamOptimizer(stepsize=0.1)

    for it in range(500):
        params, cost = opt.step_and_cost(cost_function, params)
        if it % 50 == 0:
            print (it, " iter, cost = ", cost, cost/true_value)

    minval = cost_function(params)
    return minval/true_value
# These functions are responsible for testing the solution.
random_params = np.array([np.random.rand(2)])

ops_2 = [qml.PauliX(0), qml.PauliX(1), qml.PauliX(2), qml.PauliX(3)]
coeffs_2 = [1,1,1,1]

mixer_hamiltonian = qml.Hamiltonian(coeffs_2, ops_2)

@qml.qnode(dev)
def noiseless_qaoa(params):

    for wire in range(num_wires):

        qml.Hadamard(wires = wire)

    for elem in params:

        qml.ApproxTimeEvolution(cost_hamiltonian, elem[0], 1)
        qml.ApproxTimeEvolution(mixer_hamiltonian, elem[1],1)

    return qml.expval(cost_hamiltonian)

random_params = np.array([np.random.rand(2)])

circuit_check = (np.isclose(noiseless_qaoa(random_params) - qaoa_circuit(random_params,0),0)).numpy()

def run(test_case_input: str) -> str:
    input = json.loads(test_case_input)
    output = approximation_ratio(*input)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    
    tape = qaoa_circuit.qtape
    names = [op.name for op in tape.operations]
    random_params = np.array([np.random.rand(2)])

    assert circuit_check, "qaoa_circuit is not doing what it's expected to."

    assert names.count('ApproxTimeEvolution') == 0, "Your circuit must not use the built-in PennyLane Trotterization."
     
    assert set(names) == {'DepolarizingChannel', 'RX', 'RY', 'RZ', 'CNOT'}, "Your circuit must use qml.RX, qml.RY, qml.RZ, qml.CNOT, and qml.DepolarizingChannel."

    assert solution_output > expected_output - 0.02

# These are the public test cases
test_cases = [
    ('[1, 0.003]', '0.1307'),
    ('[2,0.005]', '0.4875'),
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")    

Running test case 0 with input '[1, 0.003]'...
approximation_ratio :  1 0.003
0  iter, cost =  0.2566264324169704 -0.16324826871995815
50  iter, cost =  -0.8294279820865083 0.5276256262001361
100  iter, cost =  -0.8317574056720085 0.5291074469302411
150  iter, cost =  -0.831766079378325 0.5291129645518136
200  iter, cost =  -0.8317661060990402 0.5291129815497138
250  iter, cost =  -0.8317661068254011 0.5291129820117751
300  iter, cost =  -0.8317661068301763 0.5291129820148127
350  iter, cost =  -0.831766106830189 0.5291129820148209
400  iter, cost =  -0.8317661068301905 0.5291129820148218
450  iter, cost =  -0.8317661068301896 0.5291129820148213
Correct!
Running test case 1 with input '[2,0.005]'...
approximation_ratio :  2 0.005
0  iter, cost =  -0.6162185226782106 0.391996280480326
50  iter, cost =  -1.3741044855358011 0.8741117436397571
100  iter, cost =  -1.3764390550166783 0.8755968378382394
150  iter, cost =  -1.3764452315340743 0.8756007669181948
200  iter, cost =  -1.3764452410